<a href="https://colab.research.google.com/github/gjduart/Graph-Learning/blob/main/Simplifiying_Graph_Convolutional_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Nov  9 18:09:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.9.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 10.4 MB 4.1 MB/s 
     |████████████████████████████████| 3.7 MB 4.2 MB/s 
     |████████████████████████████████| 1.6 MB 29.7 MB/s 
     |████████████████████████████████| 870 kB 39.0 MB/s 
     |████████████████████████████████| 325 kB 5.6 MB/s 
     |████████████████████████████████| 407 kB 31.3 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.2-py3-none-any.whl size=535570 sha256=cd3652c1cd4c3541545d3b49d05c4ed51bcf532187ff527e7f1ee96bc550e75a
  Stored in directory: /root/.cache/pip/wheels/3f/08/13/2321517088bb2e95bfd0e45033bb9c923189e5b2078e0be4ef
Successfully built torch-geometric


In [ ]:
import math 
import torch
import torch.nn as nn
import torch_geometric.utils as pyg_utils
import matplotlib.pyplot as plt
import numpy as np

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import time
from time import perf_counter

In [ ]:
#torch.random.manual_seed(42) #fixando uma seed

torch.manual_seed(126) 
torch.random.manual_seed(126)
torch.cuda.manual_seed(126)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device



device(type='cuda', index=0)

In [ ]:

def load_data_planetoid(nome):
    dataset = Planetoid(root='/tmp/Planetoid', name=nome).shuffle()
    dataset = dataset[0]
    data = dataset.to(device)
    
    features = data.x
    labels = data.y 
    train_mask = data.train_mask
    print(train_mask.shape) 
    val_mask = data.val_mask
    test_mask = data.test_mask
    adj = pyg_utils.to_dense_adj(data.edge_index)
    adj = adj.reshape(features.shape[0],features.shape[0])
    print(f'Features Shape:{features.shape}')
    print(f'labels shape:  {labels.shape}')
    print(f'Adjacency shape:{adj.shape}')


    return adj,features, labels, train_mask, val_mask, test_mask

In [ ]:
adj, features, labels, idx_train, idx_val, idx_test = load_data_planetoid('cora')
adj = adj + torch.eye(adj.shape[0]).to(device)  # Add self-loops
adj

Processing...
Done!


torch.Size([2708])
Features Shape:torch.Size([2708, 1433])
labels shape:  torch.Size([2708])
Adjacency shape:torch.Size([2708, 2708])


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]], device='cuda:0')

In [ ]:
idx_train.sum(), idx_val.sum(), idx_test.sum()

(tensor(140, device='cuda:0'),
 tensor(500, device='cuda:0'),
 tensor(1000, device='cuda:0'))

In [ ]:

class SGC(nn.Module):
    def __init__(self, k, nfeat, nclass):
        super(SGC, self).__init__()
        self.k = k
        self.W = nn.Linear(nfeat, nclass, bias=True)

    def forward(self, x, adj):
        k = self.k
        for i in range(k):
            x = torch.spmm(adj, x) #Sparce matrix mutplic...
        x = self.W(x)
        return x



In [ ]:

# def train():
    
#     model.train()
#     optimizer.zero_grad()
#     loss= criterion(model()[data.train_mask], data.y[data.train_mask])
#     loss.backward()
#     optimizer.step()

def train(features,edge_index, model, opt,labels):
    model.train()
    opt.zero_grad()
    output = model(features, edge_index)
    loss = nn.functional.cross_entropy(output[idx_train], labels[idx_train])

    #loss = nn.functional.multi_margin_loss(input=output[idx_train], target=labels[idx_train],reduction='mean')


   
    loss.backward()
    opt.step()
    return loss.item()

In [ ]:
def accuracy(output, labels):
    
    preds = output.max(1)[1].type_as(labels)
    #print(preds.shape)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


In [ ]:
@torch.no_grad()
def test(features,adj, model,mask):
  
    model.eval()
    correct = 0
    output = model(features, adj)
    return accuracy(output[mask], labels[mask])

In [ ]:

epochs = 100
k = 2
lr = 0.2
wd = 0.000001


acc_total_test = []
acc_total_train = []

#Modelo

# Treinamento
iterations = 1
loss= []
for i in range(iterations):
    start_all_time = time.time()
    model = SGC(k, features.size()[1], int(labels.max().item()+1))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                              weight_decay=wd)
    model.to(device)
    for epoch in range(epochs):
          time_per_epoch =  time.time()
          train_loss = train(features,adj, model, optimizer,labels)
          loss.append(train_loss)
          train_acc = test(features,adj, model,idx_train)
          test_acc = test(features,adj, model, idx_test)

          print(f'=============iteracao{i}==================================')
          print('Epoch: {:03d}, Train Loss: {:.9f}, Train Acc: {:.7f}, Test Acc: {:.7f}, Time:{:.3f}'.format(epoch, train_loss, train_acc, test_acc, time.time()-time_per_epoch))
          if epoch==99:
            acc_total_test.append(test_acc)
            acc_total_train.append(train_acc)
    print(f'Tempo total:{time.time() - start_all_time:.3f}')      

=============iteracao0==================================
Epoch: 000, Train Loss: 3.447897196, Train Acc: 0.6071429, Test Acc: 0.5670000, Time:0.071
=============iteracao0==================================
Epoch: 001, Train Loss: 49.332828522, Train Acc: 0.5142857, Test Acc: 0.3650000, Time:0.072
=============iteracao0==================================
Epoch: 002, Train Loss: 70.257133484, Train Acc: 0.6642857, Test Acc: 0.4980000, Time:0.070
=============iteracao0==================================
Epoch: 003, Train Loss: 41.183200836, Train Acc: 0.7571429, Test Acc: 0.6080000, Time:0.064
=============iteracao0==================================
Epoch: 004, Train Loss: 29.445594788, Train Acc: 0.7857143, Test Acc: 0.6410000, Time:0.066
=============iteracao0==================================
Epoch: 005, Train Loss: 23.875240326, Train Acc: 0.8571429, Test Acc: 0.6870000, Time:0.060
=============iteracao0==================================
Epoch: 006, Train Loss: 18.446193695, Train Acc: 0

In [ ]:
acc_total_test = np.array(acc_total_test,dtype='float64')
acc_total_train = np.array(acc_total_train,dtype='float64')

train_cross_entropy_var = torch.var_mean(torch.from_numpy(acc_total_train),True)[0].item()
train_cross_entropy_mean = torch.var_mean(torch.from_numpy(acc_total_train),True)[1].item()


test_cross_entropy_var = torch.var_mean(torch.from_numpy(acc_total_test),True)[0].item()
test_cross_entropy_mean = torch.var_mean(torch.from_numpy(acc_total_test),True)[1].item()

In [ ]:
print(f'acc_train_total_variance:{train_cross_entropy_var:.4f} \nacc_train_total mean:{train_cross_entropy_mean:.3f}')
print(f'acc_test_total_variance:{test_cross_entropy_var:.4f} \nacc_test_total mean:{test_cross_entropy_mean:.3f}')


<h4><center>Training - Random Split do dataset</center></h4>
Cora: <br >
train_mask = tensor(140, device='cuda:0'), <br >
val_mask = tensor(500, device='cuda:0'),    <br >
test_mask = tensor(1000, device='cuda:0')) <br >

Citeseer: <br>
train_mask = (tensor(120, device='cuda:0'),<br >
val_mask = tensor(500, device='cuda:0'),   <br >
test_mask = tensor(1000, device='cuda:0')) <br >

pubmed: <br >
(tensor(60, device='cuda:0'),<br >
 tensor(500, device='cuda:0'),<br >
 tensor(1000, device='cuda:0'))<br >
<table>
  <tr>
    <th>Loss Function</th>
    <th>Cora</th>
    <th>Citeseer</th>
    <th>Pubmed</th>
  </tr>
  <tr>
    <td>Cross Entropy</td>
    <td>76,3% ± 0.0002</td>
    <td>56,2% ± 0.0008 </td>
    <td>74,3% ± 0.0001</td>
  </tr>
  <tr>
    <td>Hinge Loss Multiclass</td>
    <td>78,0% ± 0.0004</td>
    <td> 55,6% ± 0.0003</td>
    <td> </td>
  </tr>
</table>

In [ ]:

plt.plot(loss,'r')
plt.plot(loss_ce)
plt.show()